# Morning 1 Activities: Circuits, Measurements, & Entanglement

### Monday, June 20th 2022
### FRIB-TA Summer School
### Quantum Computing and Nuclear Few and Many-Body Physics

Welcome to your first set of hands-on activities! For the next hour, your will work in groups and gain more experience with the material covered in the morning lectures. There is a lot of material here, so don't feel compelled to finish everything. The goal is to learn something new and help each other learn as well! You can always come back to these problems later if you're interested.

Some of the skills we hope you take out of this include


*   Preparing qubit states on quantum circuits and visualizing using the Bloch sphere with Qiskit.
*   Execute quantum circuits to sample from the underlying probability distribution.
*   Explore entanglement through circuits which produce maximally entangled qubits, aka "Bell states".


Good luck! The organizers will be walking around to chat and answer questions.

Helpful links:

- [Summer school Github page](https://github.com/NuclearPhysicsWorkshops/FRIB-TASummerSchoolQuantumComputing#detailed-lecture-plan)

## Package installation

In [ ]:
try:
    import qiskit
except ImportError:
    !pip install qiskit~=0.20.0 pylatexenc --quiet

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import qiskit

version_requires = "0.20"  # Might work with other Qiskit versions, but no guaruntees :-)
assert version_requires in qiskit.__version__, print(f"Qiskit version should be >={version_requires} but is {qiskit.__version__}")

## Creating single-qubit states

A general single-qubit state can be written

$$
|\psi\rangle= \alpha |0\rangle+ \beta |1\rangle 
$$

with $|\alpha|^2 + |\beta|^2 = 1$.
We will create this state using a quantum circuit and use Qiskit to compute the amplitudes.

### Setting up a circuit

First we set up a quantum circuit in Qiskit.

In [ ]:
"""Set up a circuit in Qiskit."""
# Create a register with one qubit.
qubit = qiskit.QuantumRegister(1, name="qubit")

# Create a quantum circuit with this register.
circuit = qiskit.QuantumCircuit(qubit)

# Visualize the circuit.  Note: `print(circuit)` is also an option if you have any difficulties with plotting libraries.
circuit.draw('mpl', scale=1.5)

We can "simulate" the state vector. In this case a classical simulator stores all the information about the state vector and it is available at the end of the simulation. In contrast, a real quantum measurement cannot provide amplitudes, only bitstrings sampled with probability proportional to the mod squared amplitudes. This classical state vector simulation is useful for debugging the circuit. 

In [ ]:
"""Use Qiskit to get the final statevector of the circuit."""
# Backend to run the circuit on.
simulator = qiskit.Aer.get_backend('statevector_simulator')

# Execute the circuit.
job = qiskit.execute(circuit, simulator)

# Get the statevector.
psi = job.result().get_statevector()

# Print out the amplitudes.
for i in range(2):
    print("|" + str(i) + "⟩ amplitude: " + str(psi[i]))

We'll want to print out amplitudes several more times, so for convenience the following cell defines a function to do this. The logic is the same as above.

In [ ]:
# @markdown `def print_bracket_notation(circuit: qiskit.QuantumCircuit, precision: int = 3) -> None:`
def print_bracket_notation(circuit: qiskit.QuantumCircuit, precision: int = 3) -> None:
    """Prints the final state of the circuit in braket notation.
    
    Args:
        circuit: Circuit which prepares a quantum state.
        precision: Number of decimals for each amplitude.
    """
    psi = qiskit.execute(
        circuit, qiskit.Aer.get_backend("statevector_simulator")
    ).result().get_statevector()
    
    labels = [
      f"{str(np.round(amplitude, precision))} |{np.binary_repr(index, width=circuit.num_qubits)}⟩"
      for index, amplitude in enumerate(np.array(psi)) if np.abs(amplitude) > 0
    ]
    print("|𝜓⟩ = " + " + ".join(labels))

In [ ]:
print_bracket_notation(circuit)

### Visualizing single-qubit states on the Bloch sphere

We can visualize the state on the Bloch sphere as follows.

In [ ]:
"""Plot the qubit on the Bloch sphere."""
qiskit.visualization.plot_bloch_multivector(state=psi)

The $(x, y, z)$ coordinates on the Bloch sphere are, by definition, projections onto the Pauli $X$, $Y$, and $Z$ basis, i.e., $x = \langle \psi | X | \psi \rangle$, $y = \langle \psi | Y | \psi \rangle$, and $z = \langle \psi | Z | \psi \rangle$. You are asked to verify this below.

In [ ]:
"""Define the Pauli matrices (as `np.ndarray`s)."""
xmat = qiskit.circuit.library.XGate().to_matrix()
ymat = qiskit.circuit.library.YGate().to_matrix()
zmat = qiskit.circuit.library.ZGate().to_matrix()

In [ ]:
"""Compute the (x, y, z) Bloch sphere coordinates given a state |𝜓⟩."""
# --> Your code here.


Now use the following cell to plot the Bloch vector using your computed $(x, y, z)$ coordindates. It should match the previous Bloch vector.

In [ ]:
qiskit.visualization.plot_bloch_vector([x.real, y.real, z.real])

### Applying gates

Next, we can now apply a general transformation of the form

$$
U=\left[
\begin{array}{cc}
\cos(\theta/2) & -e^{i\lambda}\sin(\theta/2)\\
e^{i\phi}\sin(\theta/2) & e^{i\phi+i\lambda}\cos(\theta/2)
\end{array}
\right]
$$

that rotates state $|0\rangle$ to a general state

$$
|\psi\rangle=\cos(\theta/2)|0\rangle+e^{i\phi}\sin(\theta/2)|1\rangle.
$$

First, choose $\theta=\pi/2$, $\phi=\pi/2$ and $\lambda=0$ and run until the end, then return to this point,
choose $\theta=\pi/3$, $\phi=3\pi/2$ and $\lambda=0$ and rerun until the end again.


In [ ]:
"""Add the $U$ gate defined above to the circuit to prepare a general qubit state."""
# Define the parameters of the unitary.
theta, phi, lam = np.pi / 2, np.pi / 2, 0.0

# Add the unitary $U$ to the circuit.
circuit.u(theta, phi, lam, qubit)

# Visualize the circuit.
circuit.draw('mpl', scale=1.5)

Simulate  the state vector to check the amplitudes are as expected.

In [ ]:
print_bracket_notation(circuit)

Plot the new state on the Bloch sphere.

In [ ]:
"""Plot the qubit on the Bloch sphere."""
qiskit.visualization.plot_bloch_multivector(
    state=qiskit.execute(circuit, simulator).result().get_statevector()
)

As an exercise, verify that for the state 

$$
|\psi\rangle=\cos(\theta/2)|0\rangle+e^{i\phi}\sin(\theta/2)|1\rangle
$$

the Bloch vector coordinates are $(\sin \theta \cos \phi, \sin \theta \sin \phi, \cos \theta)$. The following cell plots the same Bloch vector using these coordinates.

In [ ]:
"""Plot the qubit on the Bloch sphere."""
qiskit.visualization.plot_bloch_vector(
    [np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)]
)

You can (and should!) try adding other gates to your circuit and seeing how the Bloch vector changes. For example, to add a Hadamard gate, using [`qiskit.QuantumCircuit.h`](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.h.html). What other gates are available in Qiskit? Can you define custom gates?

## Sampling from circuits

Now we add a measurement gate to the circuit. This adds a classical bit to the circuit that will store the measurement information.

In [ ]:
# Add the measurement.
circuit.measure_all()

# Visualize the circuit.
circuit.draw('mpl', scale=1.5)

We can now execute the circuit on a quantum simulator which samples bitstrings (instead of returning the final statevector).

In [ ]:
"""Execute the circuit on a sampler."""
# Set the number of measurement shots (samples). Vary this number!
shots = 10_000

# Simulate the circuit and get the results.
sampler = qiskit.Aer.get_backend('qasm_simulator')
job = qiskit.execute(circuit, sampler, shots=shots)
result = job.result()

# Get the measurement counts and visualize them.
measurements = result.get_counts(circuit)
print(measurements)
qiskit.visualization.plot_histogram(measurements)

How close are the sampled bitstrings to $|\alpha|^2$ and $|\beta|^2$?

## Measuring expectation values of Paulis

Given a Hamiltonian written as a linear combination of Paulis $H = \sum_i a_i \sigma_i$, the energy of a state $|\psi\rangle$ is

$$
\langle \psi | H | \psi \rangle = \sum_i a_i \langle \psi | \sigma_i | \psi \rangle .
$$

A common subroutine in quantum algorithms is computing the $\langle \psi | \sigma_i | \psi \rangle$ terms on a quantum computer, then using a classical computer to multiply by $a_i$ and add the results.

Here we show how to compute $\langle \psi | Z | \psi \rangle$ from measurements for an arbitrary single-qubit state $|\psi\rangle$. The general case will be handled in upcoming hands-on sessions.

In [ ]:
"""Prepare any single-qubit state |𝜓⟩ in a circuit."""
circuit = qiskit.QuantumCircuit(1)
circuit.h(0)
circuit.t(0)
circuit.sxdg(0)
# Add any gates here.

circuit.draw('mpl', scale=1.5)

We know how to compute the state vector from a circuit. Let's use this to exactly compute $\langle \psi | Z | \psi \rangle$ to compare our result to. Note that this is just for comparison - in practice we won't have access to the wavefunction $|\psi\rangle$.

In [ ]:
"""Compute the expectation value "exactly" (without sampling)."""
psi = np.array(qiskit.execute(circuit, simulator).result().get_statevector())

expectation_value_from_state_vector = psi.conj().T @ zmat @ psi
print("⟨𝜓|𝑍|𝜓⟩ =", expectation_value_from_state_vector)

Now let's sample from this circuit.

In [ ]:
"""Add measurements to sample."""
circuit.measure_all()  # This is in a separate cell because it will add measurements every time it is run.

In [ ]:
"""Measure the qubit."""
# Set the number of shots.
shots = 1_000

# Get the measurements from the sampler.
measurements = qiskit.execute(circuit, sampler, shots=shots).result().get_counts()
print(measurements)

How do we go from counts of bitstrings to a floating point value for $\langle \psi | Z | \psi \rangle$? Note that

$$
Z = |0\rangle \langle 0 | - |1\rangle \langle 1 |
$$

so that

$$
\langle \psi | Z | \psi \rangle = \langle \psi | 0 \rangle \langle 0 | \psi \rangle - \langle \psi | 1 \rangle \langle 1 | \psi \rangle .
$$

But these terms are the probability of measuring $0$ and $1$, respectively:

$$
\langle \psi | Z | \psi \rangle = p(0) - p(1) .
$$

When we sample, we don't exactly know $p(0)$, but we estimate it by how frequently it is measured compared to the total number of bitstrings measured. 

Thus, we have a way to estimate floating point expectation values by measuring bitstrings. The next cell implements this.

In [ ]:
"""Compute the expectation value from sampling. Read above to see why this works."""
expectation_value_from_samples = (measurements.get("0", 0.0) - measurements.get("1", 0.0)) / shots

# Display the result.
print("⟨𝜓|𝑍|𝜓⟩ ≈", expectation_value_from_samples)

# Compare it to the result without sampling.
print("\n\nWithout sampling, ⟨𝜓|𝑍|𝜓⟩ =", expectation_value_from_state_vector)

To consider:

- How close is the estimate from sampling to the exact value as a function of the number of shots?
- Try adding different gates to your circuit and computing the expectation value again.
- [Challenge] How could you modify the above to compute $\langle \psi | X | \psi \rangle$ and $\langle \psi | Y | \psi \rangle$? (Hint: Try to relate $X$ and $Y$ to $Z$.)

## Measuring correlations in entangled states

Consider the four Bell states in a two-qubit system

$$
|\beta_{00}\rangle=\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)
$$
$$
|\beta_{01}\rangle=\frac{1}{\sqrt{2}}(|01\rangle+|10\rangle)
$$
$$
|\beta_{10}\rangle=\frac{1}{\sqrt{2}}(|00\rangle-|11\rangle)
$$
$$
|\beta_{11}\rangle=\frac{1}{\sqrt{2}}(|01\rangle-|10\rangle)
$$

In relation to the [Einstein, Podolsky, Rosen (1935) paper](https://cds.cern.ch/record/405662/files/PhysRev.47.777.pdf) they are also called EPR states or EPR pairs. (Note that EPR originally formulated the problem with the position and momentum, and later a simpler reformulation was offered by Bohm for two-level systems, e.g. qubits, electron spin, photon polarization.)

We will prepare these states in a circuit and measure the qubits to see how the measurement outcomes are correlated.

### Base case: Uncorrelated state

First, let's measure a tensor product state as a "base case".

In [ ]:
"""Tensor product state: Uncorrelated measurements."""
qc = qiskit.QuantumCircuit(2)
qc.h(0)
qc.h(1)
qc.measure_all()

qc.draw("mpl", scale=1.5)

In [ ]:
"""Sample bitstrings and display the results."""
measurements = qiskit.execute(qc, sampler, shots=shots).result().get_counts()
qiskit.visualization.plot_histogram(measurements)

Note there is no correlation in the measured bitstrings `00`, `01`, `10`, and `11`. We can visualize this in a different way as follows.

In [ ]:
# @markdown `def measure(...)`: Function to return measurements as lists of `int`s.
from typing import List, Tuple


def measure(circuit: qiskit.QuantumCircuit, shots: int = 1000) -> Tuple[List[int], List[int]]:
    """Returns measurements from "Alice" and "Bob", i.e., measurements on each
    qubit.

    Args:
        circuit: A two-qubit quantum circuit with measurements.
        shots: Number of samples (shots) to take.
    """
    if circuit.num_qubits != 2:
        raise ValueError(
            f"Requires `circuit` to have two qubits, but circuit has "
            f"{circuit.num_qubits} qubits."
        )

    job = qiskit.execute(circuit, sampler, shots=shots, memory=True)
    bits = job.result().get_memory(circuit)
    
    alice_bits = []
    bob_bits = []

    # Split the bit values into two arrays, convert to integer.
    for alice, bob in bits:
        alice_bits.append(int(alice))
        bob_bits.append(int(bob))

    return alice_bits, bob_bits

This function returns the measurements on each qubit (Alice and Bob's measurements). Let's plot the results in the unit square.

In [ ]:
plt.plot(*measure(qc), "o", ms=10, mec="black", alpha=0.5);

This is another way of visually seeing no correlations in the measurement outcomes.

### Entangled state $|\beta_{00}\rangle$

Now we prepare the state $|\beta_{00}\rangle$ in a circuit.

In [ ]:
"""|𝛽00⟩ state."""
qc = qiskit.QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw("mpl", scale=1.5)

As above, we measure this state and visualize the outcomes.

In [ ]:
measurements = qiskit.execute(qc, sampler, shots=shots).result().get_counts()
qiskit.visualization.plot_histogram(measurements)

Note now the measurements are strongly correlated. We can also visualize this in the unit square as above.

In [ ]:
plt.plot(*measure(qc), "o", ms=10, mec="black", alpha=0.5);

Your task is to repeat this for the remaining three EPR states!

### Entangled state $|\beta_{01}\rangle$

In [ ]:
"""|𝛽01⟩ state."""
# --> Your code here.


In [ ]:
measurements = qiskit.execute(qc, sampler, shots=shots).result().get_counts()
qiskit.visualization.plot_histogram(measurements)

In [ ]:
plt.plot(*measure(qc), "o", ms=10, mec="black", alpha=0.5);

### Entangled state $|\beta_{10}\rangle$

In [ ]:
"""|𝛽10⟩ state."""
# --> Your code here.


In [ ]:
measurements = qiskit.execute(qc, sampler, shots=shots).result().get_counts()
qiskit.visualization.plot_histogram(measurements)

In [ ]:
plt.plot(*measure(qc), "o", ms=10, mec="black", alpha=0.5);

### Entangled state $|\beta_{11}\rangle$

In [ ]:
"""|𝛽11⟩ state."""
# --> Your code here.


In [ ]:
measurements = qiskit.execute(qc, sampler, shots=shots).result().get_counts()
qiskit.visualization.plot_histogram(measurements)

In [ ]:
plt.plot(*measure(qc), "o", ms=10, mec="black", alpha=0.5);

Congrats on making it through this notebook!

## Challenge

Done early?

- Try measuring the four EPR states on hardware, i.e., a real quantum computer, not a simulator. Do you still see correlated measurements? Are there any bitstrings that weren't there before? Why or why not?
- These types of measurements are similar to those done in a [CHSH experiment](https://qiskit.org/textbook/ch-demos/chsh.html). Check out the linked Qiskit tutorial to experimentally disprove local hidden variable theories.